<a href="https://colab.research.google.com/github/suwatoh/Python-learning/blob/main/131_%E5%9B%BD%E9%9A%9B%E5%8C%96.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

国際化
======

国際化と地域化
--------------

プログラムを世界の複数の地域に対応するようにしたい場合、必要に応じて機能的な変更や拡張を行うというやり方では、開発や保守に多くの時間と費用がかかる。

そこで、プログラムの本質的な機能と、複数の地域に対応する機能を分離し、プログラムに機能的な変更や拡張を加えることなく複数の地域に適合できるようにするための設計が考案された。そのような設計を**国際化**（internationalization）といい、 i18n と略される（internationalization の先頭の i と語尾の n の間に 18 文字があることに起因する）。主なものは以下の通り。

  * Unicode への対応
  * 右横書き言語への対応
  * 翻訳機能の組み込み
  * タイムゾーン対応の組み込み
  * 通貨情報の組み込み
  * 地域特有の表記への書式化

i18n の設計に基づいてプログラムを特定の地域に対応させるための実装は**地域化**（localization）と呼ばれ、L10N と略される（localization の先頭の l と語尾の n の間に 10 文字があることに起因する）。地域化には、縦書き、ルビへの対応など、地域固有の追加機能を含むこともある。

現代の OS は国際化されており、特定の地域への対応が簡単な設定だけで実現されている。例えば、Unix 系 OS の場合、環境変数などで言語の設定を行う。 Unix 系 OS 上で動く国際化されたプログラムは、環境変数などの設定を参照して地域の言語を決定している。

Python は言語レベルで Unicode に対応し、また、標準ライブラリの `datetime` モジュールでタイムゾーンに対応し、標準ライブラリの `locale` モジュールで通貨情報と地域特有の表記への書式化に対応している。

翻訳機能
--------

クロスプラットフォームで開発されるプログラムでは、プログラム中の文字列を翻訳する機能に [GNU gettext](https://www.gnu.org/software/gettext/) ライブラリを利用することが広まっている。 GNU gettext では、以下のように翻訳機能をサポートする。

  1. ソースコードの中で翻訳対象とする文字列を `_("...")` で囲んでマークする。
  2. マークづけをしたファイルに `xgettext` コマンドを用いて翻訳可能な全ての文字列のリストを保持する .pot ファイルを生成する。
  3. 必要により .pot ファイルにコメントを付ける。コメントは翻訳に役立つ情報とする。ソースコード中の文字列の直前にコメントを置くと、そのコメントが .pot ファイルに引き継がれる。
  4. .pot ファイルを入力として `msginit` コマンドを実行し、言語コードの名前を持つ .po ファイルを生成する。日本語なら `ja.po`、韓国語なら `ko.po`、中国語（簡体字）なら `zh-CN.po`、スペイン語なら `es.po` というようになる。ファイル名を指定することも可能。
  5. 各 .po ファイルの中の文字列のリストに対して翻訳作業を行っていく。手作業またはツールにより .po ファイルを編集する。
  6. .poファイルを `msgfmt` コマンドで .mo バイナリファイルにコンパイルする。.mo ファイルは通常 `locale/<言語コード>_<国コード>/LC_MESSAGES` という言語ごとの専用のディレクトリに配置される。日本語なら `locale/ja_JP/LC_MESSAGES` である。プログラム本体と翻訳リソースをパッケージにして配布する。
  7. OS は環境変数より翻訳リソースを検索し（.mo ファイル中に該当言語のリソースがありさえすれば）、プログラムにその言語による表示を行わせることができる。

Python 標準ライブラリの `gettext` モジュールは、上記の過程 7 の機能に対する Python インターフェースを提供する。

``` python
gettext.translation(domain, localedir=None, languages=None, class_=None, fallback=False)
```

この関数は、`<localedir>/<language>/LC_MESSAGES/<domain>.mo` ファイルから翻訳リソースを探索する翻訳オブジェクトを返す。`<language>` はシーケンスである `languages` の要素である。`localedir` 引数が省略された場合、標準のシステムロケールディレクトリが使われる。`languages` 引数が省略された場合、環境変数 `LANGUAGE`、`LC_ALL`、`LC_MESSAGES`、および `LANG` が検索される。

`class_` 引数に翻訳オブジェクトの型を指定できる。指定されていない場合は `gettext.GNUTranslations` クラスとなる。

.mo ファイルが見つからなかった場合、 `fallback` 引数が `False`（デフォルト値）ならこの関数は `OSError` を送出し、`fallback` 引数が `True` なら `gettext.NullTranslations` インスタンスが返される。 `gettext.NullTranslations` クラスは、翻訳クラスを実装するのに使える基本的なインターフェースを提供している。 `gettext.GNUTranslations` クラスも `gettext.NullTranslations` のサブクラスである。 `gettext.NullTranslations` クラスのインターフェース自体は、未翻訳の文字列を出力する。

翻訳オブジェクトの主なメソッドは次のとおり。

| メソッド | 機能 | 戻り値 |
|:---|:---|:---|
| `gettext(message)` | `message` のローカライズされた翻訳を返す | `str` |
| `install(names=None)` | このオブジェクトの `gettext()` メソッドを組み込み名前空間 `__builtins__` にインストールし、変数 `_` に束縛する。`names` は<br />インストールする別のメソッドをシーケンスで指定するための引数 | `None` |

翻訳オブジェクトの `install()` メソッドの使用は便利な方法ではあるが、組み込み名前空間 `__builtins__` にインストールするため、その影響はプログラム全体に及ぶことに注意する。モジュールのグローバルな名前空間にインストールするには、このメソッドの代わりに次のコードを使用して `_(...)` を使用できるようにする必要がある:

``` python
import gettext
t = gettext.translation('mymodule', ...)
_ = t.gettext
```

以下のファイル配置を持つプロジェクトで、`gettext` モジュールによる翻訳を実現する例を示す。

``` text
my-project/
    my_package/
        __init__.py
        __main__.py
    locale/
        ja_JP/
            LC_MESSAGES/
                messages.po
                messages.mo
        messages.pot
```

ここでは、`__main__.py` で翻訳機能を実装する。

`__main__.py`:  
``` python
import gettext
import os

import my_package

def init_translation():
    # 翻訳ファイルを配置するディレクトリ
    path_to_locale_dir = os.path.abspath(
        os.path.join(
            os.path.dirname(__file__),
            '../locale'
        )
    )

    # 翻訳用クラスの設定
    translater = gettext.translation(
        'messages',                   # domain: 辞書ファイルの名前
        localedir=path_to_locale_dir, # 辞書ファイル配置ディレクトリ
        languages=['ja_JP'],          # 翻訳に使用する言語
        fallback=True                 # .moファイルが見つからなかった時は未翻訳の文字列を出力
    )

    # gettext() メソッドを組み込み名前空間で _ に束縛する
    translater.install()

# プログラムを実行
if __name__ = '__main__':
    init_translation()
    my_package.main()
```

翻訳対象の文字列をマーク付けしておく。

`__init__.py`:  
``` python
def main():
    print(_('Hello, World!'))
```

.pot ファイルを生成する:  
``` shell
PS> xgettext -o locale/messages.pot my_package/__init__.py
```

.poファイルの作成:  
``` shell
PS> msginit -l ja_JP -o locale/ja_JP/LC_MESSAGES/messages.po -i locale/messages.pot
```

.poファイルを開いて、翻訳を入力する:  
``` text
msgid "Hello, World!"
msgstr "こんにちは、世界！"
```

.moファイルへのコンパイル:  

``` shell
PS> msgfmt locale/ja_JP/LC_MESSAGES/messages.po -o locale/ja_JP/LC_MESSAGES/messages.mo
```

すべてが設定されたら、プログラムを実行して、指定した言語の翻訳が適用されているか確認する。

``` shell
PS> python -m my_package
こんにちは、世界！
```